In [1]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import pandas as pd
# numpy
import numpy as np
# random
from random import shuffle
# classifier
from sklearn.linear_model import LogisticRegression

C:\Users\scott\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# inspecting data size

In [2]:
NegTrainDataPath = "..\\CleanData\\train-neg.txt"
PosTrainDataPath = "..\\CleanData\\train-pos.txt"
UnLabelTrainDataPath = "..\\CleanData\\train-unsup.txt"
testDataPath = "..\\CleanData\\test.txt"

In [3]:
NegLabelTrainDataFrame = pd.read_csv(NegTrainDataPath,header=None)
PosLabelTrainDataFrame = pd.read_csv(PosTrainDataPath,header=None)
UnLabelTrainDataFrame = pd.read_csv(UnLabelTrainDataPath,header=None)
TestDataFrame = pd.read_csv(testDataPath,header=None)

In [4]:
print("NegTrainData = ",NegLabelTrainDataFrame.shape[0])
print("PosTrainData = ",PosLabelTrainDataFrame.shape[0])
print("UnLabelTrainData = ",UnLabelTrainDataFrame.shape[0])
print("TestData = ",TestDataFrame.shape[0])

NegTrainData =  12500
PosTrainData =  12500
UnLabelTrainData =  50000
TestData =  25000


# Train model and save to disk

Attention: here I omit some very important step, such as how to choose super parameter, you can check ``Doc2Vec.py`` to see my settings

for example, ``mini cout`` is a subtle parameter, I suggest it should be 30 to 50, since each movie occurs 30 times

``size`` is another importatne paramter, More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 400.

more tech to select `Doc2vec` parameter can be found in here:https://www.kaggle.com/c/word2vec-nlp-tutorial#part-2-word-vectors

In [2]:
sources = {'CleanData\\test.txt': 'TEST', 'CleanData\\train-neg.txt': 'TRAIN_NEG',
            'CleanData\\train-pos.txt': 'TRAIN_POS', 'CleanData\\train-unsup.txt': 'TRAIN_UNS'}
test = Doc2Vec.Train(sources)
test.process()
test.save('.\\Model\\imdb.d2v')

## loadmodel

In [2]:
model = Doc2Vec.load('../Persistence/Model/imdb.d2v')

##  a sample vector 

In [3]:
model['TRAIN_NEG_0']

array([-2.3791196 , -0.9334    ,  1.0988426 , -0.94970757, -1.8186786 ,
       -1.6068125 , -0.8571677 , -0.71717787,  0.07672118,  0.3074862 ,
       -1.2175689 ,  0.5106428 , -0.1872679 ,  1.1860161 , -2.0082247 ,
       -1.118874  ,  0.5909139 , -0.27789843,  0.5407293 ,  1.4340234 ,
        0.36014515, -0.56291395, -0.7012361 , -0.8333109 , -1.3679545 ,
       -0.50061065, -0.11031017,  1.1482604 , -0.10441066, -2.6325662 ,
        0.5159839 , -1.5029888 ,  0.525387  , -0.8092677 ,  1.0439361 ,
       -1.8843384 , -1.6593565 , -3.3668585 ,  1.1564244 ,  2.047468  ,
       -0.80236375,  2.2020767 ,  1.1480349 , -0.65203184,  0.85826904,
        0.3046908 ,  2.2514524 ,  2.5198383 ,  1.4709095 , -0.7628401 ,
       -0.8293866 ,  0.4131753 ,  2.7910335 , -0.9634716 ,  0.47208372,
        1.3176486 , -0.454331  ,  0.8937898 ,  0.8185652 , -1.3318874 ,
        0.00654519,  3.2685728 ,  1.5720541 ,  1.6603485 , -0.29937273,
       -1.1018872 ,  1.2067581 ,  1.50389   , -0.5250043 ,  1.21

## Inspecting the Model

### similar words

check sentiment words, like ``good``,``suck``,find the most similar words

In [8]:
model.wv.most_similar('good')

[('great', 0.5707298517227173),
 ('decent', 0.5701993703842163),
 ('bad', 0.4825226664543152),
 ('fine', 0.4680075943470001),
 ('nice', 0.4640767574310303),
 ('terrific', 0.40220221877098083),
 ('cool', 0.3792990744113922),
 ('solid', 0.375804603099823),
 ('impressive', 0.3720175623893738),
 ('weak', 0.37139537930488586)]

In [9]:
model.wv.most_similar('bad')

[('terrible', 0.5165358781814575),
 ('horrible', 0.5109846591949463),
 ('awful', 0.48885127902030945),
 ('good', 0.4825226664543152),
 ('poor', 0.42794835567474365),
 ('stupid', 0.4101775288581848),
 ('lousy', 0.39594411849975586),
 ('inept', 0.37407028675079346),
 ('horrid', 0.36204835772514343),
 ('atrocious', 0.35902562737464905)]

seem pretty good, since our analysis depends on the sentiment words, so their relation is rather important

### match words

to check model robust

In [6]:
model.wv.doesnt_match("man woman child kitchen".split())

'kitchen'

In [7]:
model.wv.most_similar(positive=['woman','boy'],negative=['man'],topn=10)

[('girl', 0.639438271522522),
 ('sister', 0.3268493413925171),
 ('daughter', 0.32179757952690125),
 ('female', 0.3057842254638672),
 ('child', 0.30202263593673706),
 ('boys', 0.28385084867477417),
 ('girls', 0.27956512570381165),
 ('kid', 0.27465373277664185),
 ('babe', 0.26127684116363525),
 ('sexual', 0.2572239339351654)]

### model corpus

In [12]:
len(model.wv.vectors_norm )

19312

### TensorBoard

In [3]:
import sys, os
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim.models import Word2Vec
import tensorflow as tf
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector

def visualize(model, output_path):
    meta_file = "w2x_metadata.tsv"
    placeholder = np.zeros((len(model.wv.index2word), model.wv.vector_size))

    with open(os.path.join(output_path,meta_file), 'wb') as file_metadata:
        for i, word in enumerate(model.wv.index2word):
            placeholder[i] = model[word]
            # temporary solution for https://github.com/tensorflow/tensorflow/issues/9094
            if word == '':
                print("Emply Line, should replecaed by any thing else, or will cause a bug of tensorboard")
                file_metadata.write("{0}".format('<Empty Line>').encode('utf-8') + b'\n')
            else:
                file_metadata.write("{0}".format(word).encode('utf-8') + b'\n')

    # define the model without training
    sess = tf.InteractiveSession()

    embedding = tf.Variable(placeholder, trainable = False, name = 'w2x_metadata')
    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(output_path, sess.graph)

    # adding into projector
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = 'w2x_metadata'
    embed.metadata_path = meta_file

    # Specify the width and height of a single thumbnail.
    projector.visualize_embeddings(writer, config)
    saver.save(sess, os.path.join(output_path,'w2x_metadata.ckpt'))
    print('Run `tensorboard --logdir={0}` to run visualize result on tensorboard'.format(output_path))

if __name__ == "__main__":
    model = Word2Vec.load("..\\Model\\imdb.d2v")
    visualize(model,"..\\Model\\visual")

Run `tensorboard --logdir=..\Model\visual` to run visualize result on tensorboard


### Run `tensorboard --logdir=..\Model\visual` to run visualize result on tensorboard

![52776363609](http://wx4.sinaimg.cn/mw690/0060lm7Tly1fs1iq0ghwzj31gq0pt12q.jpg)

### review "suck" using euclidean  distance

![52776398234](http://wx1.sinaimg.cn/mw690/0060lm7Tly1fs1iozlqgej30v10m4mym.jpg)

# save vector for LSTM 

for LSTM use, we need to convert document to a list of vector, I choose to vectorize firt `500` words as their feature, so every document has `500` feature, each feature is a `400` length vector

In [8]:
def SaveDataForRNN(self, feature=500):
        """
        save X_train,Y_train,X_test to ``./Persistence/``\n
        X_train.shape = (25000,feature_size,model.vector.size)\n
        Y_train.shape = (25000,2)  [1,0] for ``positive`` [0,1] for ``negative``\n
        too big to return, read data from ``./Persistence``
        """
        sources = {'CleanData\\test.txt': 'TEST', 'CleanData\\train-neg.txt': 'TRAIN_NEG',
                   'CleanData\\train-pos.txt': 'TRAIN_POS', 'CleanData\\train-unsup.txt': 'TRAIN_UNS'}

        # dict sentences_dict["TRAIN_NEG_0"] = ["a","b",..."last word"]
        sentences = LabeledLineSentence(sources)
        sentences_array = sentences.to_array()
        sentences_dict = {}
        for i in range(0, len(sentences_array)):
            sentences_dict[sentences_array[i][1][0]] = sentences_array[i][0]

        # Index2word is a list that contains the names of the words in
        # the model's vocabulary. Convert it to a set, for speed
        index2word_set = set(self.model.wv.index2word)

        # when short of word, using zero instead
        empty_word = np.zeros(self.model.vector_size,dtype='float16')


        X_train = np.zeros((25000, feature, self.model.vector_size),dtype='float16')
        Y_train = np.zeros(25000)



        # get first 1000 feature vectors from 1000 words
        for i in range(12500):
            prefix_train_pos = 'TRAIN_POS_' + str(i)
            prefix_train_neg = 'TRAIN_NEG_' + str(i)
            # length of document
            len1 = len(sentences_dict[prefix_train_pos])
            len2 = len(sentences_dict[prefix_train_neg])
            cout = j = 0
            # for pos document
            while cout < feature:
                if j < len1:
                    word = sentences_dict[prefix_train_pos][j]
                    if word in index2word_set:
                        X_train[i, cout, :] = self.model[word]
                        cout += 1
                else:
                    X_train[i, cout, :] = empty_word
                    cout += 1
                j += 1

            # reset j/cout, for neg document
            cout = j = 0
            while cout < feature:
                if j < len2:
                    word = sentences_dict[prefix_train_neg][j]
                    if word in index2word_set:
                        X_train[12500+i, cout, :] = self.model[word]
                        cout += 1
                else:
                    X_train[12500+i, cout, :] = empty_word
                    cout += 1
                j += 1

            Y_train[i] = 1
            Y_train[12500 + i] = 0

        with open('./Persistence/LSTM/X_train.pickle', 'wb') as f:
            pickle.dump(X_train, f, protocol=4)
            f.close()

        with open('./Persistence/LSTM/Y_train.pickle', 'wb') as f:
            pickle.dump(Y_train, f, protocol=4)
            f.close()

        X_test =np.zeros((25000, feature, self.model.vector_size),dtype='float16')

        for i in range(25000):
            prefix_test = 'TEST_' + str(i)
            len1 = len(sentences_dict[prefix_test])
            cout = j = 0
            while cout < feature:
                if j < len1:
                    word = sentences_dict[prefix_test][j]
                    if word in index2word_set:
                        X_test[i, cout, :] = self.model[word]
                        cout += 1
                else:
                    X_test[i, cout, :] = empty_word
                    cout += 1
                j += 1

        with open('./Persistence/LSTM/X_test.pickle', 'wb') as f:
            pickle.dump(X_test, f, protocol=4)
            f.close()

        print("save narray success to ./Persistence")

        # return X_train,Y_train,X_test